# Deep Dive: Non-Markovian Dynamics with Memory

**Level**: Intermediate  
**Prerequisites**: `spiral_time_intro.ipynb`

---

## Overview

This notebook provides a comprehensive exploration of non-Markovian dynamics in the spiral-time framework:

1. **Memory kernels**: Different types and their properties
2. **Operational signatures**: How to detect memory experimentally
3. **Comparison with environmental memory**: Critical distinctions
4. **Process dynamics**: Multi-time correlations

By the end, you'll understand:
- How different memory kernels affect dynamics
- Quantitative measures of non-Markovianity
- How to distinguish intrinsic from environmental memory

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.linalg import expm, sqrtm
import ipywidgets as widgets
from IPython.display import display, HTML

np.random.seed(2026)
plt.style.use('seaborn-v0_8-whitegrid')

print("✓ Environment ready")

---

## Part 1: Memory Kernels

### The Memory Kernel K(t-τ)

From Equation 18 in the paper:
$$\dot{\rho}(t) = \mathcal{L}[\rho(t)] + \int_0^t K(t-\tau) \rho(\tau) d\tau$$

### Types of Memory Kernels

1. **Exponential**: $K(\Delta t) = \varepsilon e^{-\gamma \Delta t}$ - Fast decay
2. **Power-Law**: $K(\Delta t) = \varepsilon / (1 + \gamma \Delta t)^\alpha$ - Slow decay
3. **Oscillatory**: $K(\Delta t) = \varepsilon e^{-\gamma \Delta t} \cos(\omega \Delta t)$ - Information backflow

In [ ]:
class MemoryKernel:
    @staticmethod
    def exponential(dt, epsilon=0.1, gamma=1.0):
        return epsilon * np.exp(-gamma * dt)
    
    @staticmethod
    def power_law(dt, epsilon=0.1, gamma=1.0, alpha=1.5):
        return epsilon / (1 + gamma * dt)**alpha
    
    @staticmethod
    def oscillatory(dt, epsilon=0.1, gamma=0.5, omega=3.0):
        return epsilon * np.exp(-gamma * dt) * np.cos(omega * dt)

# Visualize kernels
dt_array = np.linspace(0, 10, 500)
K_exp = [MemoryKernel.exponential(dt) for dt in dt_array]
K_pow = [MemoryKernel.power_law(dt) for dt in dt_array]
K_osc = [MemoryKernel.oscillatory(dt) for dt in dt_array]

plt.figure(figsize=(12, 4))
plt.plot(dt_array, K_exp, 'b-', label='Exponential', linewidth=2)
plt.plot(dt_array, K_pow, 'r-', label='Power-law', linewidth=2)
plt.plot(dt_array, K_osc, 'g-', label='Oscillatory', linewidth=2)
plt.xlabel('Time lag Δt')
plt.ylabel('K(Δt)')
plt.title('Memory Kernel Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Key differences:")
print("  - Exponential: Fast decay, finite memory")
print("  - Power-law: Slow decay, long-range correlations")
print("  - Oscillatory: Can be negative, information backflow")

---

## Part 2: Quantifying Non-Markovianity

### Trace Distance Measure

For Markovian dynamics, trace distance never increases:
$$D(\rho_1(t), \rho_2(t)) = \frac{1}{2}\|\rho_1(t) - \rho_2(t)\|_1$$

**Non-Markovian signature**: Distance increases → information backflow

In [ ]:
def trace_distance(rho1, rho2):
    diff = rho1 - rho2
    eigenvalues = np.linalg.eigvalsh(diff @ diff.conj().T)
    return 0.5 * np.sum(np.sqrt(np.abs(eigenvalues)))

# Simulate and test
rho1 = np.array([[1, 0], [0, 0]], dtype=complex)
rho2 = np.array([[0, 0], [0, 1]], dtype=complex)

print(f"Initial trace distance: {trace_distance(rho1, rho2):.4f}")
print("\nNon-Markovian dynamics show distance increases over time")

---

## Part 3: Intrinsic vs Environmental Memory

### Environmental Memory
- **Origin**: System-bath coupling
- **State-dependent** kernel
- Can be eliminated by isolation
- Finite-rank process tensors

### Intrinsic Spiral-Time Memory
- **Origin**: Fundamental temporal structure
- **State-independent** kernel
- Persists in isolation
- Unbounded process tensor rank

In [ ]:
def simulate_intrinsic_memory(rho_init, epsilon=0.1, t_final=10.0, dt=0.05):
    H_S = 0.5 * np.array([[1, 0], [0, -1]], dtype=complex)
    rho = rho_init.copy()
    history = [(0.0, rho.copy())]
    times = [0.0]
    
    t = 0.0
    while t < t_final:
        drho_H = -1j * (H_S @ rho - rho @ H_S)
        drho_memory = np.zeros_like(rho)
        
        for t_past, rho_past in history:
            K = epsilon * np.exp(-(t - t_past))
            drho_memory += K * rho_past * dt
        
        rho = rho + (drho_H + drho_memory) * dt
        rho = (rho + rho.conj().T) / 2
        rho = rho / np.trace(rho)
        
        t += dt
        history.append((t, rho.copy()))
        times.append(t)
    
    return times, history

# Test
rho_init = 0.5 * np.array([[1, 1], [1, 1]], dtype=complex)
times, history = simulate_intrinsic_memory(rho_init)
coherence = [np.abs(h[1][0,1]) for h in history]

plt.figure(figsize=(10, 4))
plt.plot(times, coherence, 'b-', linewidth=2)
plt.xlabel('Time t')
plt.ylabel('Coherence |ρ₀₁|')
plt.title('Intrinsic Memory Evolution')
plt.grid(True, alpha=0.3)
plt.show()

print("Intrinsic memory: State-independent kernel")
print("Test: Vary initial state - kernel remains unchanged")

---

## Part 4: Multi-Time Correlations

### Two-Time Correlators
$$C(t_1, t_2) = \langle Q(t_1) Q(t_2) \rangle$$

### Leggett-Garg Parameter
$$K_3 = C(t_1,t_2) + C(t_2,t_3) - C(t_1,t_3)$$

Classical bound: $-3 \leq K_3 \leq 1$

In [ ]:
def compute_correlator(times, history, observable, t1_idx, t2_idx):
    rho1 = history[t1_idx][1]
    rho2 = history[t2_idx][1]
    Q1 = np.real(np.trace(observable @ rho1))
    Q2 = np.real(np.trace(observable @ rho2))
    return Q1 * Q2

print("Multi-time correlations reveal memory structure")
print("Non-Markovian: Persistent correlations at large time separations")

---

## Summary

### Key Takeaways

1. **Memory kernels** determine non-Markovian character
2. **Trace distance** quantifies information backflow
3. **Intrinsic vs environmental**: State-independence is key
4. **Multi-time correlations** reveal memory structure

### Next Steps

Continue to:
- `cp_divisibility_tutorial.ipynb`: Falsification criterion
- `protocol_examples.ipynb`: Experimental implementations

---

## Exercises

In [ ]:
# Exercise 1: Implement Gaussian kernel
# K(Δt) = ε * exp(-(Δt/τ)²)

# Your code here:


# Exercise 2: Compute BLP non-Markovianity measure
# N_BLP = ∫ max(0, dD/dt) dt

# Your code here:


---

## References

- **Paper Section 3**: Non-Markovian Dynamics
- **Paper Section 10**: Experimental Discrimination
- Breuer et al. (2009). *Phys. Rev. Lett.* **103**, 210401
- de Vega & Alonso (2017). *Rev. Mod. Phys.* **89**, 015001